# Assignment 5: Web scrapping elections from the JNE page
The script should not give an error. Any mistake will be consider as 0.


Group 3

-Fátima Trujillo Quiñe
-Reynaldo Padilla Milla
-Claudia Córdova Yamauchi
-Mauricio Flores Jiménez
-Vania Aspilcueta Serey

# 1. Calling the libraries

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pandas as pd
import os

# 2. Setting the driver and the page

In [3]:
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()))
driver

# Configuramos el tamaño de la ventana y el zoom
driver.maximize_window()
driver.execute_script("document.body.style.zoom='100%'")

# 3. Indicating the web page and actions

In [5]:
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)

# We'll click on the first [SELECCIONE], the one for the elections type
elections = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div').click()
time.sleep(5)

# Next, we choose (click on) ELECCIONES PRESIDENCIALES
elections = driver.find_element(By.XPATH , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()
time.sleep(5)

# Then, click on the [SELECCIONE] on the right (under Elección)
elections = driver.find_element(By.XPATH , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div').click()
time.sleep(5)

# Now, we show the elements of the second menu
elections_list = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]')
time.sleep(5)

# Move on to create a list of elements representing election options
options_elections_list = list(elections_list.find_elements(By.CLASS_NAME, 'item'))

# Create a dictionary mapping the text of each option to the element itself
dict_options = {option.text: option for option in options_elections_list}

dict_options

{'[SELECCIONE]': <selenium.webdriver.remote.webelement.WebElement (session="ae80863491697fcf18c84a3535e08194", element="f.4B3BEB52D11553DB694DF64FEAF6189E.d.371A83B801576E63FC369B35B0E492C7.e.33")>,
 'PRESIDENCIAL 2021 - 2DA VUELTA': <selenium.webdriver.remote.webelement.WebElement (session="ae80863491697fcf18c84a3535e08194", element="f.4B3BEB52D11553DB694DF64FEAF6189E.d.371A83B801576E63FC369B35B0E492C7.e.34")>,
 'PRESIDENCIAL 2021': <selenium.webdriver.remote.webelement.WebElement (session="ae80863491697fcf18c84a3535e08194", element="f.4B3BEB52D11553DB694DF64FEAF6189E.d.371A83B801576E63FC369B35B0E492C7.e.35")>,
 'PRESIDENCIAL 2016 - 2DA VUELTA': <selenium.webdriver.remote.webelement.WebElement (session="ae80863491697fcf18c84a3535e08194", element="f.4B3BEB52D11553DB694DF64FEAF6189E.d.371A83B801576E63FC369B35B0E492C7.e.36")>,
 'PRESIDENCIAL 2016': <selenium.webdriver.remote.webelement.WebElement (session="ae80863491697fcf18c84a3535e08194", element="f.4B3BEB52D11553DB694DF64FEAF6189E.d.3

# 4. Establishing the actions to get the data from each electoral process

In [7]:
# Create an empty dataframe for our scrapping
df_results = pd.DataFrame()

# Iterate over the keys of the dictionary, which are the election options (2021's run-off, 2021's first round, 2016's run-off, etc.)
# We broaden the waiting time to avoid any problems related to un-uploaded information
for election_txt in list(dict_options.keys())[1:]:  # Excluding "SELECCIONAR"
    election = dict_options[election_txt]
    election.click()

    # Wait until the election data button is clickable and click it
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button'))
    ).click()

    # Wait until the "Candidates and results" tab is clickable and click it
    WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a'))
    ).click()

    # Wait until the table is present
    table = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]'))
    )

    # Read the HTML of the table
    table_html = table.get_attribute('innerHTML')
    table = pd.read_html(table_html)

    # Create a dataframe
    df = table[0]

    # Create a new DataFrame containing only the relevant columns and add the election year
    df = df[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']].copy()
    df.insert(0, 'ELECCIONES', election_txt)

    # Concatenate results in df_results
    df_results = pd.concat([df_results, df], ignore_index=True)

    # Go back to start over
    driver.back()
    time.sleep(15)

    driver.back()
    time.sleep(15)
    
    # Click on the second list to choose the election
    WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div'))
    ).click()
    time.sleep(15)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x006E8923+23283]
	(No symbol) [0x006AE934]
	(No symbol) [0x005E0733]
	(No symbol) [0x0062326F]
	(No symbol) [0x006234AB]
	(No symbol) [0x0065EE42]
	(No symbol) [0x00644464]
	(No symbol) [0x0065CB8D]
	(No symbol) [0x006441B6]
	(No symbol) [0x00618017]
	(No symbol) [0x0061890D]
	GetHandleVerifier [0x007DA5F3+1013699]
	GetHandleVerifier [0x007E3E4C+1052700]
	GetHandleVerifier [0x007DD4B4+1025668]
	GetHandleVerifier [0x0070EA2B+179195]
	(No symbol) [0x006B6833]
	(No symbol) [0x006B3198]
	(No symbol) [0x006B3337]
	(No symbol) [0x006AB4BE]
	BaseThreadInitThunk [0x76ECFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776780CE+286]
	RtlGetAppContainerNamedObjectPath [0x7767809E+238]


# 5. Verifying the outcome

In [9]:
#We call the dataframe in order to see the structure
df_results


,ELECCIONES,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 2021 - 2DA VUELTA,PARTIDO POLITICO NACIONAL PERU LIBRE,8836380
1,PRESIDENCIAL 2021 - 2DA VUELTA,FUERZA POPULAR,8792117
2,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS EN BLANCO,121489
3,PRESIDENCIAL 2021 - 2DA VUELTA,VOTOS NULOS,1106816


# 6. Exporting the results as an Excel file

In [ ]:
# Finally, we create the name of the xlsx file
file_name = "df_results.xlsx"

# We place it in the same route where we have been working
current_dir = os.getcwd()

# Afterwards, we link the route and the file name in order to create the file path 
file_path = os.path.join(current_dir, file_name)

#At last, we save the dataframe "df_results". As it is shown, index=False is preventing that the dataframe index saves as a new column in Excel file.
df_results.to_excel(file_path, index=False)
